In [4]:
import vertexai

from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool

from IPython.display import Markdown


REGION = "asia-south1"
vertexai.init(location=REGION)

EMBEDDING_MODEL = "publishers/google/models/text-embedding-005"
embedding_model_config = rag.EmbeddingModelConfig(publisher_model=EMBEDDING_MODEL)

rag_corpus = rag.create_corpus(
    display_name="RAG Corpus",
    embedding_model_config=embedding_model_config,
)

BUCKET_NAME = "rag-grounding-bucket"

INPUT_GCS_BUCKET = (
    f"gs://{BUCKET_NAME}/rag/QuickBite_FoodDeliveryPlatform_CustomerServiceManual.pdf"
)

rag_file = rag.import_files(
    corpus_name=rag_corpus.name,
    paths=[INPUT_GCS_BUCKET],
    chunk_size=1000,
    chunk_overlap=100,
    max_embedding_requests_per_min=900,
)

rag.list_files(corpus_name=rag_corpus.name)

rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_corpora=[rag_corpus.name],
            similarity_top_k=10,
            vector_distance_threshold=0.5,
        ),
    )
)

#5.Integrate retrieval tool with Gemini to create a RAG-Gemini model
rag_gemini_model = GenerativeModel(
    "gemini-2.5-flash",
    tools=[rag_retrieval_tool]
)

response = rag_gemini_model.generate_content("What is the protocol for handling an incorrect order delivery?")

display(Markdown(response.text))

response_1 = rag_gemini_model.generate_content("How do you explain the refund policy to a customer dissatisfied with their order?")

display(Markdown(response_1.text))

If your QuickBite order is incorrect, you should contact customer service immediately. The customer service team will investigate the issue by checking order details and delivery logs. A resolution will then be provided, which may include a full or partial refund, free delivery, or a discount. If the issue is not resolved by the representative, it can be escalated to a supervisor.

When a customer is dissatisfied with their order, you should first listen actively and empathetically, apologizing for any inconvenience. Then, you can explain that QuickBite issues refunds under specific conditions: if the customer receives an incorrect order (full or partial refund), if the order is delayed beyond the promised time (refund or partial refund), or if the food arrives damaged (refund or replacement).

Ensure you offer a solution that aligns with QuickBite's policy, which may include a refund, free delivery, or a discount for a future purchase. Refund requests are typically processed within 3-5 business days, and the customer will be updated on the status of their request. If the issue cannot be resolved, it should be escalated to a supervisor.